In [49]:
print("all ok")

all ok


In [50]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [51]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [53]:
extracted_data = load_pdf("../data")

Ignoring wrong pointing object 2 65536 (offset 0)
Ignoring wrong pointing object 14 65536 (offset 0)
Ignoring wrong pointing object 18 65536 (offset 0)
Ignoring wrong pointing object 22 65536 (offset 0)
Ignoring wrong pointing object 26 65536 (offset 0)
Ignoring wrong pointing object 30 65536 (offset 0)
Ignoring wrong pointing object 34 65536 (offset 0)
Ignoring wrong pointing object 2 65536 (offset 0)


In [13]:
# extracted_data

In [54]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [55]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5139


In [56]:
from langchain.embeddings import HuggingFaceEmbeddings
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [57]:

embeddings = download_hugging_face_embeddings()

In [58]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [59]:
query_result=embeddings.embed_query("Hello world")
print("length",len(query_result))

length 384


In [20]:
# query_result

In [60]:
import os

from dotenv import load_dotenv

In [61]:

load_dotenv()

True

In [62]:
import pinecone

In [63]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install --upgrade pinecone pinecone-plugin-assistant

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-pinecone 0.2.3 requires pinecone<6.0.0,>=5.4.0, but you have pinecone 6.0.1 which is incompatible.



  Using cached pinecone-6.0.1-py3-none-any.whl.metadata (8.8 kB)
Using cached pinecone-6.0.1-py3-none-any.whl (421 kB)
  Attempting uninstall: pinecone
    Found existing installation: pinecone 5.4.2
    Uninstalling pinecone-5.4.2:
      Successfully uninstalled pinecone-5.4.2


In [64]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

In [27]:
from pinecone import Pinecone
from pinecone import ServerlessSpec

In [28]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [65]:

pc = Pinecone(api_key=PINECONE_API_KEY)

In [66]:
index_name = "debate-ai"

In [32]:

# pc.create_index(
#     name=index_name,
#     dimension=384, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )

In [67]:
pip install langchain_pinecone

Note: you may need to restart the kernel to use updated packages.


In [68]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [69]:
docsearch

In [70]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [71]:
docsearch

In [72]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [73]:
retrieved_docs = retriever.invoke("What is Acne?")

In [74]:
retrieved_docs 

[Document(id='6f1e90dc-c485-4bec-8175-c572761e5f66', metadata={'author': 'Shaw, Warren Choate, 1887-', 'creationdate': '2012-12-12T14:04:06+00:00', 'creator': 'Digitized by the Internet Archive', 'keywords': 'http://archive.org/details/artofdebate00shaw', 'moddate': '2012-12-12T14:45:36+00:00', 'page': 26.0, 'page_label': '11', 'producer': 'Recoded by LuraDocument PDF v2.53', 'source': '..\\data\\artofdebate00shaw.pdf', 'title': 'The art of debate', 'total_pages': 488.0}, page_content='eakers.Sucharequirementmeansthatthepropositionshouldnotinvolveaproblemthatisbeyondtheintellectualcapacityofthespeakerstohandle.Inotherwords,thepropositionshouldnotdemandofthespeakersatechnicalknowledgethattheyareunabletoacquirewithinareasonableperiodofpreparationfordebate.Tofulfillthisrequirementmostsatisfactorily,thepropositionshouldusuallyinvolveadiscussionofmatterthatisalreadysomewhatfamiliartothespeakers.'),
 Document(id='bb62a1b3-a1f4-47a7-a6e8-75ac856602e2', metadata={'author': 'Shaw, Warren Choate

In [75]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [76]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [77]:
import google.generativeai as genai

In [ ]:
genai.configure(api_key= )

In [79]:
def query_gemini_with_pinecone(user_query):
    # Perform similarity search in Pinecone
    search_results = docsearch.similarity_search(user_query, k=2)

    # Extract context from search results
    context_texts = [doc.page_content for doc in search_results]
    context_combined = '\n\n'.join(context_texts)

    # Generate answer using Gemini
    model = genai.GenerativeModel('gemini-pro')
    prompt = PROMPT.format(context=context_combined, question=user_query)
    response = model.generate_content(prompt)

    return response.text

# Test the retrieval and generation flow
user_query = "what is debate?"
response = query_gemini_with_pinecone(user_query)
print("User Query:", user_query)
print("Response:", response)

ResourceExhausted: 429 Resource has been exhausted (e.g. check quota).

In [ ]:
search_results = docsearch.similarity_search(user_query, k=2)

    # Extract context from search results
context_texts = [doc.page_content for doc in search_results]
context_combined = '\n\n'.join(context_texts)

In [44]:
model = genai.GenerativeModel('gemini-pro')
prompt = PROMPT.format(context=context_combined, question=user_query)
response = model.generate_content(prompt)

NameError: name 'context_combined' is not defined

In [ ]:
from langchain_community.llms import CTransformers

In [ ]:
# !pwd
# llm=CTransformers(model="../models/llama_model_1.bin",  
#                   model_type="llama",
#                   config={'max_new_tokens':512,
#                           'temperature':0.8})

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x000002A612122510>, model='../models/laama.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

In [ ]:
from langchain.schema import Document

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\Asus\AppData\Local\Temp\ipykernel_26744\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Debating is an art form, just like playing chess, you need to learn the moves, mastering different techniques and strategies, and practice makes perfect!


ProtocolError: Failed to connect; did you specify the correct index name?